<a href="https://colab.research.google.com/github/xiaoyufan/nbme/blob/main/baseline_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NBME Baseline DeBERTa

## Configurations

In [108]:
class Config:
  batch_size = 16
  device_name = 'tpu'
  epochs = 1
  input_dir = '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Preprocessed'
  mode = 'dev'
  model = 'microsoft/deberta-base'
  output_dir = '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output'
  sequence_max_length = 466

## Packages

In [109]:
!pip install transformers
!pip install tokenizers

!pip install --force-reinstall git+https://github.com/xiaoyufan/nbme.git

!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

  Cloning https://github.com/xiaoyufan/nbme.git to /tmp/pip-req-build-_jef2515
  Running command git clone -q https://github.com/xiaoyufan/nbme.git /tmp/pip-req-build-_jef2515
  Created wheel for nbme: filename=nbme-1.0-py3-none-any.whl size=3129 sha256=43b242e2446f27b7efdbcdc7d588cef1d9ac9698c35568bbeb46ca9a38b343f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-vuzf4uhs/wheels/e2/dc/dd/b061e30220e414c0a096e4fbaddd405c1b526d1453d9a447e3
Successfully built nbme
  Attempting uninstall: nbme
    Found existing installation: nbme 1.0
    Uninstalling nbme-1.0:
      Successfully uninstalled nbme-1.0


  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl (152.9 MB)


In [110]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from ast import literal_eval
from nbme_utils.location import location_to_span
from transformers import AutoConfig, AutoModel, AutoTokenizer, PreTrainedTokenizer
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

## Device

In [111]:
import os

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if Config.device_name == 'tpu':
  assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

  import torch_xla
  import torch_xla.core.xla_model as xm

  DEVICE = xm.xla_device()

DEVICE

device(type='xla', index=1)

## Tokenizer

In [112]:
tokenizer = AutoTokenizer.from_pretrained(Config.model)
tokenizer.save_pretrained(f'{Config.output_dir}/tokenizer')

('/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/vocab.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/merges.txt',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/tokenizer.json')

## Data Loading

### Defining Dataset

In [113]:
train = pd.read_csv(f'{Config.input_dir}/train.csv')
valid = pd.read_csv(f'{Config.input_dir}/validate.csv')
train.shape, valid.shape

((11342, 8), (2958, 8))

In [114]:
if Config.mode == 'dev':
  train = train.sample(n=800, random_state=0).reset_index(drop=True)
  valid = valid.sample(n=200, random_state=0).reset_index(drop=True)
train.shape, valid.shape

((800, 8), (200, 8))

In [115]:
train.head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text
0,00225_008,0,225,8,"['drinks coffee', 'drinks energy drinks']","['461 467;483 489', '461 467;506 519']",17 y/o previously healthy male here with heart...,Caffeine-use
1,81979_803,8,81979,803,[],[],67yo woman presents with 3 weeks of difficulty...,Auditory-hallucination-once
2,31800_311,3,31800,311,[],[],HPI: Mr. Hamilton is a 35 year old male who pr...,No-blood-in-stool
3,20158_216,2,20158,216,['44'],['22 24'],"Ms Dolores Montgmery, 44 yo F c/o Irregular me...",44-year
4,21591_201,2,21591,201,[],[],Dolores Montgomery is a 44 year old female wit...,Last-Pap-smear-I-year-ago


In [116]:
def generate_labels(encoded: PreTrainedTokenizer, sample: pd.DataFrame):
  labels = torch.zeros(len(encoded['input_ids']))

  location_spans = location_to_span(literal_eval(sample['location']))

  for idx, (seq_id, offsets)in enumerate(zip(encoded['sequence_ids'],
                                             encoded['offset_mapping'])):
    # None for special tokens added around or between sequences,
    # 0 for tokens corresponding to words in the first sequence,
    # 1 for tokens corresponding to words in the second sequence when a pair of sequences was jointly encoded.
    # Labels are generated from patient notes, which are encoded as the first sequence.
    if seq_id != 0:
      labels[idx] = -1
      continue

    subtoken_start, subtoken_end = offsets

    if any([subtoken_start >= location_start and subtoken_end <= location_end
            for location_start, location_end in location_spans]):
      labels[idx] = 1

  return labels

class NBMEDataset(Dataset):
  def __init__(self, data: pd.DataFrame, tokenizer: AutoTokenizer, config: Config):
    self.data = data
    self.tokenizer = tokenizer
    self.config = config

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx: int):
    sample = self.data.iloc[idx]

    encoded = self.tokenizer(
      sample['pn_history'],
      sample['feature_text'],
      # TODO: Compute max length of sequences
      max_length=self.config.sequence_max_length,
      padding='max_length',
      return_offsets_mapping=True,
    )
    encoded['sequence_ids'] = np.array(encoded.sequence_ids()).astype('float16')

    y_true = generate_labels(encoded, sample)

    x = encoded.copy()
    skip_tensor_conversion = ['offset_mapping']
    x = {k: (torch.tensor(v, dtype=torch.long) if k not in skip_tensor_conversion else v)
         for k, v in x.items()}

    return x, y_true

In [117]:
train_dataset = NBMEDataset(train, tokenizer, Config)
valid_dataset = NBMEDataset(valid, tokenizer, Config)

### Dataset Loader

In [118]:
train_loader = DataLoader(
    train_dataset,
    batch_size=Config.batch_size,
    shuffle=True,)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=Config.batch_size,
    shuffle=False,)

In [119]:
x, y_true = next(iter(train_loader))

## Model

In [124]:
class NBMEDebertaBaseline(nn.Module):
  def __init__(self):
    super().__init__()

    config = AutoConfig.from_pretrained(Config.model, output_hidden_states=True)
    self.model = AutoModel.from_pretrained(Config.model, config=config)

    self.fc = nn.Linear(config.hidden_size, 1)

  def forward(self, input_ids, attention_mask):
    hidden_states = self.model(input_ids=input_ids, attention_mask=attention_mask)
    logits = self.fc(hidden_states[0])
    logits = torch.sigmoid(logits)
    return logits

In [125]:
model = NBMEDebertaBaseline().to(DEVICE)

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Training

## Testing

In [126]:
def test_fn(model, device):
  model.eval()
  preds = []
  offsets = []
  seq_ids = []
  
  for batch in tqdm(valid_loader):
      x, y_pred = batch
      input_ids = x['input_ids'].to(device)
      attention_mask = x['attention_mask'].to(device)
      offset_mapping = x['offset_mapping']
      sequence_ids = x['sequence_ids']
  
      logits = model(input_ids, attention_mask)
      
      preds.append(logits.detach().cpu().numpy())
      offsets.append(offset_mapping.numpy())
      seq_ids.append(sequence_ids.numpy())
  
  preds = np.concatenate(preds, axis=0)
  offsets = np.concatenate(offsets, axis=0)
  seq_ids = np.concatenate(seq_ids, axis=0)

In [127]:
test_fn(model, DEVICE)

  0%|          | 0/13 [00:00<?, ?it/s]

AttributeError: ignored